<h3>1. 쿠팡의 주문 목록 리스트를 크롤링</h3>

In [13]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [36]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import subprocess

# 크롬 실행 파일 위치
subprocess.Popen(r'C:\Program Files\Google\Chrome\Application\chrome.exe --remote-debugging-port=9222') # 맥에선 주석처리
options = webdriver.ChromeOptions()
options.add_experimental_option("debuggerAddress", "127.0.0.1:9222") # 맥에선 주석처리
driver = webdriver.Chrome(options=options)


# 로그인 웹사이트 주소
driver.get('https://login.coupang.com/login/login.pang?')
driver.implicitly_wait(3)

try:
    # 로그인
    username_input = driver.find_element(By.ID, 'login-email-input')
    password_input = driver.find_element(By.ID, 'login-password-input')

    username_input.send_keys("your_email") # your_email
    password_input.send_keys("your_password") # your_password
    username_input.submit()
    time.sleep(3)

# 로그인 되어있을 경우 바로 크롤링
finally:
    # 주문목록 웹사이트 주소
    driver.get("https://mc.coupang.com/ssr/desktop/order/list")
    driver.implicitly_wait(3)

# 크롤링
# "주문상세보기" aria-hidden으로 접근 불가
result = []
div = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "sc-fimazj-0")))
for d in div:
    data = [{"shop_name" : "쿠팡"}]
    
    # 날짜 추출
    date = d.find_element(By.CLASS_NAME, "sc-abukv2-1").text[:-3] # "주문" 글자 제외
    data.append({"date" : date})

    items = []

    # 상품 이름 추출
    product = []
    inner_div = d.find_elements(By.CLASS_NAME, "sc-755zt3-1")
    for i in inner_div:
        product.append(i.text)
   
    # 상품 수량 추출
    quantity = []
    inner_div = d.find_elements(By.CLASS_NAME, "sc-755zt3-1")
    for i in inner_div:
        quantity.append(i.text)

    # 상품 가격 추출
    price = []
    inner_div = d.find_elements(By.CLASS_NAME, "sc-755zt3-1")
    for i in inner_div:
        price.append(i.text)
   
    for x, y, z in zip(product, quantity, price):
        items.append({"name" : x})
        items.append({"quantity" : y})
        items.append({"price" : price})
    data.append({"items" : items})
    result.append({"data" : data})    
for _ in result:
    print(_)

# Chromedriver 종료
driver.quit()

{'data': [{'shop_name': '쿠팡'}, {'date': '2024. 9. 21'}, {'items': [{'name': '티아시아키친 스파이시 비프 마살라 커리, 170g, 8개'}, {'quantity': '티아시아키친 스파이시 비프 마살라 커리, 170g, 8개'}, {'price': ['티아시아키친 스파이시 비프 마살라 커리, 170g, 8개', '티아시아키친 치킨 마크니 커리 전자레인지용, 170g, 8개']}, {'name': '티아시아키친 치킨 마크니 커리 전자레인지용, 170g, 8개'}, {'quantity': '티아시아키친 치킨 마크니 커리 전자레인지용, 170g, 8개'}, {'price': ['티아시아키친 스파이시 비프 마살라 커리, 170g, 8개', '티아시아키친 치킨 마크니 커리 전자레인지용, 170g, 8개']}]}]}
{'data': [{'shop_name': '쿠팡'}, {'date': '2024. 8. 16'}, {'items': [{'name': 'QCY Melobuds Pro HT08 ANC 노이즈 캔슬링 멜로버즈 프로 무선 블루투스 이어폰 앱연동, HT08 블랙'}, {'quantity': 'QCY Melobuds Pro HT08 ANC 노이즈 캔슬링 멜로버즈 프로 무선 블루투스 이어폰 앱연동, HT08 블랙'}, {'price': ['QCY Melobuds Pro HT08 ANC 노이즈 캔슬링 멜로버즈 프로 무선 블루투스 이어폰 앱연동, HT08 블랙']}]}]}
{'data': [{'shop_name': '쿠팡'}, {'date': '2024. 8. 6'}, {'items': [{'name': '라보에이치 두피강화 클리닉 탈모증상완화 샴푸, 400ml, 1개'}, {'quantity': '라보에이치 두피강화 클리닉 탈모증상완화 샴푸, 400ml, 1개'}, {'price': ['라보에이치 두피강화 클리닉 탈모증상완화 샴푸, 400ml, 1개']}]}]}
{'data': [{'shop_name': '쿠팡'

<h3>2. 크롤링한 정보 저장 DB 설계</h3>

        크롤링한 상품 상세 정보는 정보간 관계를 분석하기 보단 key, value 쌍의 정보 확인 용도로 DB를 구성하는 게 적합하므로 대표적인 NoSQL DB인 MongoDB가 적합합니다.

In [23]:
pip install pymongo

Note: you may need to restart the kernel to use updated packages.


In [42]:
import pymongo
  
uri = "<your mongodb driver>"

client = MongoClient(uri, server_api=ServerApi('1'))
mydb = client['Coupang']
mycol = mydb['Crawling']

# mycol.delete_many({})

for r in result:
    x =  mycol.insert_one({"Result" : r})

In [ ]:
# 상품 상세 페이지에서 다음과 같은 정보도 추출 가능
# driver.get(find_elements(By.CLASS_NAME, sc-1uwk3m0-0).get_attribute('href')) # 상품 상세 페이지 주소

# 상품 무게 당 가격
# find_elements(By.CLASS_NAME, "sc-755zt3-1")

# 상품 별점
# find_elements(By.CLASS_NAME, "sdp-review__average__total-star__info-orange")

# 상품 리뷰 수
# find_elements(By.CLASS_NAME, sdp-review__average__total-star__info-count)

# 판매자 정보
# find_elements(By.CLASS_NAME, prod-sale-vendor-name)

<h3>3. 고객정보와 상품정보의 연관 분석</h3>

        고객의 구매시간, 성별, 연령을 조사하여 동일한 성격의 그룹과 비슷한 품목을 구입하는 지 분석하여 비슷하다면 해당 상품들을 추천할 수 있습니다. 마케팅정보수신에 동의한 경우 마케팅 알림이 온 물건을 구매했는 지를 분석해 마케팅 빈도 및 마케팅 대상 상품을 조절할 수 있습니다.

<h3>4. F&B 상품과 비 F&B 상품 구분</h3>

        F&B 상품은 무게 당 가격(100g당 가격 등)이 제시되어 있어 이를 통해 비 F&B 상품과 구분가능합니다. 코드로 분석 시 비 F&B 상품 페이지에서는 "sc-755zt3-1" 값을 가진 클래스를 찾을 수 없습니다.